In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
# import seaborn as sns
plt.rc('font', family='Malgun Gothic')
plt.rc('axes', unicode_minus=False)
from IPython.display import set_matplotlib_formats
set_matplotlib_formats('retina')
pd.set_option('display.max_rows', None)

import time
import re
import folium
import warnings
warnings.filterwarnings('ignore')

표본수 측정 참고 사이트: https://www.nownsurvey.com/calculator/  
* **https://www.surveymonkey.com/mp/sample-size-calculator/**
* **https://techntalk.tistory.com/entry/%ED%91%9C%EB%B3%B8%EC%A1%B0%EC%82%AC-%EC%98%A4%EC%B0%A8%EB%B2%94%EC%9C%84margin-of-error-%ED%98%B9%EC%9D%80-%ED%91%9C%EB%B3%B8%EC%98%A4%EC%B0%A8%EB%B2%94%EC%9C%84%EC%9D%98-%EC%9D%98%EB%AF%B8-%EC%A7%80%EC%A7%80%EC%9C%A8%EC%9D%98-%EC%8B%A0%EB%A2%B0%EA%B5%AC%EA%B0%84%EA%B3%BC-%EC%83%98%ED%94%8C%ED%81%AC%EA%B8%B0-%EA%B2%B0%EC%A0%95%EB%AC%B8%EC%A0%9C**

# 표본수 측정

* 2019년 기준 대한민국 국민 수 : 51710000  
* 신뢰수준 95%  
* 표본오차 5%로 표본 수 측정  
  
[사례] 선거철이 되면 우리는 다음과 같은 기사를 흔히 접하게 된다. “후보 지지율 설문조사에서 A후보는 40%, B후보는 30%의 지지율을 얻었다. 이번 조사는 전국 성인남녀 천명을 대상으로 무작위 전화 면접조사로 실시되었고 신뢰수준 95%에서 표본오차는 ±3.0%포인트이다.”

(의문) 여기서 ‘신뢰수준 95%에서 표본오차 3.0%포인트’란 말의 의미는 무엇일까?

(해석) 위와 같은 동일한 형태의 여론조사를 100번 실시한다면 95번은 A후보가 40%에서 ±3.0% 인 37% ~ 43%, B후보는 30%에서 ±3.0% 인 27% ~ 33% 사이의 지지율을 얻을 것으로 기대된다는 의미이다.  

즉, 이 조사를 95% 믿을 수 있다는 뜻이 아니라 같은 조사를 100번 하면 표본오차 범위 내에서 동일한 결과가 나올 횟수가 95번이라는 의미다. 표본오차는 표본 크기가 커질수록 줄어들고, 작아지면 늘어난다.

$$n = \frac{\frac{Z^2\cdot p(1-p)}{e^2}}{1+(\frac{Z^2\cdot p(1-p)}{e^2N})}$$

In [9]:
# 95% 신뢰수준의 z-score = 1.96
N = 51710000
Z = 1.96
e = 0.05
p = 0.5

denominaotr = 1 + (Z**2 * (p*(1-p)) / (e**2 * N))
numerator = (Z**2 * (p*(1-p))) / e**2
n = numerator / denominaotr
n

384.1571460489416

In [10]:
# # 전체 메타데이터는 데이터 정합성이 떨어져 데이터 목록(191건) 크롤링하기로 결정
# df = pd.read_csv('./전국민방위대피시설표준데이터.csv', encoding='cp949')
# print(df.shape)
# print(df.isnull().sum())
# df.head()

# Data 크롤링

In [7]:
driver = webdriver.Chrome('C:/Users/seob6/Desktop/chromedriver/chromedriver.exe')

# 로그인
driver.get('https://www.data.go.kr/uim/login/loginView.do')
time.sleep(1)
id = driver.find_element_by_id('mberId')
pw = driver.find_element_by_id('pswrd')
id.send_keys('seob6615')
pw.send_keys('!ichthus13!')
driver.find_element_by_class_name('btn-login').click()

# 대피시설 링크
driver.get('https://www.data.go.kr/data/15021098/standard.do')

# 1페이지 데이터 다운로드
driver.find_element_by_xpath('//*[@id="contents"]/div[2]/nav/strong').click()
time.sleep(1.5)
data_list = driver.find_elements_by_css_selector('div.data-list li > div.tit > a')  
for i in data_list:
    driver.implicitly_wait(3)
    i.click()
    driver.find_element_by_xpath('//*[@id="layer_data_infomation"]/div[2]/table/tbody/tr[7]/td/div/div/div[2]/a').click()
    time.sleep(1)
    driver.find_element_by_xpath('//*[@id="layer_data_infomation"]/div[3]/div/a').click()   

# 페이지 돌면서 지역별 데이터 다운로드
while 1:
    for i in np.arange(3, 13):
        driver.find_element_by_xpath(f'//*[@id="contents"]/div[2]/nav/a[{i}]').click()     
        time.sleep(1)
        
        # 데이터 5개 목록
        data_list = driver.find_elements_by_css_selector('div.data-list li > div.tit > a')  
        print(len(data_list))
        breaker = False

        try: 
            if (len(data_list) <= 5) and (len(data_list) >= 2):
                for i in data_list:
                    driver.implicitly_wait(3)
                    i.click()
                    driver.find_element_by_xpath('//*[@id="layer_data_infomation"]/div[2]/table/tbody/tr[7]/td/div/div/div[2]/a').click()
                    time.sleep(1)
                    driver.find_element_by_xpath('//*[@id="layer_data_infomation"]/div[3]/div/a').click()
            elif len(data_list) == 1:
                for i in data_list:
                    driver.implicitly_wait(3)
                    i.click()
                    driver.find_element_by_xpath('//*[@id="layer_data_infomation"]/div[2]/table/tbody/tr[7]/td/div/div/div[2]/a').click()
                    time.sleep(1)
                    driver.find_element_by_xpath('//*[@id="layer_data_infomation"]/div[3]/div/a').click()
                raise Exception('No More Data')
            elif len(data_list) == 0:
                raise Exception('No More Data')

        except Exception as e:
            print(e)
            breaker = True
            break
    if breaker == True:
        break

5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
1
No More Data
